In [1]:
def parsePosition(line):
    return tuple(map(lambda x: int(x), line.split(",")))

def getInput(fileName):
    with open(fileName, "r") as file:
        input = []
        for line in map(lambda x: x.strip("\n"), file.readlines()):
            if len(line) == 0:
                continue

            if line.startswith("---"):
                scanner = []
                input.append(scanner)            
            else:
                scanner.append(parsePosition(line))

        return input

In [2]:
def getOrientations():
    for axis in range(3):
        for direction in [1, -1]:
            for rotation in range(4):
                forwards = [0, 0, 0]
                forwards[axis] = direction
                yield {
                    'axis': tuple(forwards),
                    'angle': rotation
                }

In [3]:
def rotateOnce(position, axis):
    for index, direction in enumerate(axis):
        if direction != 0:
            break

    match index:
        case 0: return (position[0], position[2] * -direction, position[1] * direction)
        case 1: return (position[2] * direction, position[1], position[0] * -direction)
        case 2: return (position[1] * -direction, position[0] * direction, position[2])

def orientatePosition(position, orientation, reverse=False):

    if isinstance(orientation, list):
        if len(orientation) == 0:
            return position
        return orientatePosition(orientatePosition(position, orientation[0], reverse), orientation[1:], reverse)

    result = position
    
    if reverse:
        # Rotate x axis so it alines with normalized x axis
        if orientation['axis'][0] == -1:
            result = rotateOnce(result, (0, 0, -1))
            result = rotateOnce(result, (0, 0, -1))
        if orientation['axis'][1] != 0:
            result = rotateOnce(result, (0, 0, -orientation['axis'][1]))
        elif orientation['axis'][2] != 0:
            result = rotateOnce(result, (0, -orientation['axis'][2], 0))

        for _ in range(4 - (orientation['angle'] % 4)):
            result = rotateOnce(result, orientation['axis'])
        return result

    # Set rotation of the position to 0
    for _ in range(orientation['angle'] % 4):
        result = rotateOnce(result, orientation['axis'])

    # Rotate x axis so it alines with normalized x axis
    if orientation['axis'][0] == -1:
        result = rotateOnce(result, (0, 0, 1))
        result = rotateOnce(result, (0, 0, 1))
    if orientation['axis'][1] != 0:
        result = rotateOnce(result, (0, 0, orientation['axis'][1]))
    elif orientation['axis'][2] != 0:
        result = rotateOnce(result, (0, orientation['axis'][2], 0))

    return result

In [4]:
# Code to test the orentation code works

input = getInput('example-2.txt')
origin = input[0]

def countMatches(a, b):
    result = 0
    for i in range(min(len(a), len(b))):
        if a[i] == b[i]:
            result += 1

    return result

testResults = {}
for index, scanner in enumerate(input[1:]):
    for orientation in getOrientations():
        orientated = list(map(lambda x: orientatePosition(x, orientation), scanner))
        matchCount = countMatches(origin, orientated)
        if matchCount == len(origin):
            testResults[index] = orientation

for testResult in testResults.items():
    print(testResult)
print(f'{len(testResults)} out of {len(input[1:])} test beacons passed')

(0, {'axis': (-1, 0, 0), 'angle': 1})
(1, {'axis': (0, 0, 1), 'angle': 0})
(2, {'axis': (0, 0, 1), 'angle': 2})
(3, {'axis': (0, 0, -1), 'angle': 1})
4 out of 4 test beacons passed


In [5]:
def getMatches(origin, beacons):
    result = []
    for beacon in beacons:
        if beacon in origin:
            result.append(beacon)
    return result

def calculateRelativePositions(scanners):
    def getAbsolutePosition(scannerIndex, previous=None):
        parents = transforms[scannerIndex]
        if 0 in parents:
            offset, scannerRotation, matches = parents[0]
            return offset, [scannerRotation], matches

        for parentIndex in parents:
            if previous != None and parentIndex in previous:
                continue
            newPrevious = set([scannerIndex])
            if previous != None:
                newPrevious.update(previous)
            result = getAbsolutePosition(parentIndex, newPrevious)
            if result == None:
                continue

            offset, scannerRotation, matches = parents[parentIndex]
            parentOffset, parentRotations, _ = result
            rotatedOffset = orientatePosition(offset, parentRotations)
            absoluteOffset = (parentOffset[0] + rotatedOffset[0], parentOffset[1] + rotatedOffset[1], parentOffset[2] + rotatedOffset[2])
            return absoluteOffset, [scannerRotation] + parentRotations, list(map(lambda x: orientatePosition(x, parentRotations), matches))
            
    transforms = {}
    for i in range(len(scanners)):
        transforms[i] = {}

    for originIndex in range(len(scanners)):
        origin = scanners[originIndex]
        originSet = set(origin)
        for index, beacons in enumerate(scanners):
            if index == originIndex:
                continue
            for orientation in getOrientations():
                orientatedBeacons = list(map(lambda x: orientatePosition(x, orientation), beacons))
                for beacon in orientatedBeacons:
                    for targetBeacon in origin:
                        offset = (targetBeacon[0] - beacon[0], targetBeacon[1] - beacon[1], targetBeacon[2] - beacon[2])
                        offsetBeacons = list(map(lambda x: (x[0] + offset[0], x[1] + offset[1], x[2] + offset[2]), orientatedBeacons))
                        matches = getMatches(originSet, offsetBeacons)
                        if len(matches) >= 12:
                            transforms[index][originIndex] = (offset, orientation, matches)
                            break
    positions = {}
    for scanner in range(len(scanners)):
        positions[scanner] = getAbsolutePosition(scanner)
    return positions


In [6]:
def getBeaconCount(input):
    relativePositions = calculateRelativePositions(input)
    beacons = set()
    for i, scanner in enumerate(input):
        offset, rotations, _ = relativePositions[i]
        rotatedBeaconPositions = map(lambda x: orientatePosition(x, rotations), scanner)
        absoluteBeaconPositions = map(lambda x: (x[0] + offset[0], x[1] + offset[1], x[2] + offset[2]), rotatedBeaconPositions)
        beacons.update(absoluteBeaconPositions)
    return len(beacons)

In [7]:
input = getInput('example.txt')

results = calculateRelativePositions(input)
expected = {
    0: (0, 0, 0),
    1: (68, -1246, -43),
    2: (1105, -1205, 1229),
    3: (-92, -2380, -20),
    4: (-20, -1133, 1061)
}

items = list(results.items())
items.sort(key=lambda x: x[0])
for key, value in items:
    print(f'{key}: {str("None" if value is None else value[0]): <20} {"Correct" if value is not None and expected[key] == value[0] else "Should be: " + str(expected[key])}')

0: (0, 0, 0)            Correct
1: (68, -1246, -43)     Correct
2: (1105, -1205, 1229)  Correct
3: (-92, -2380, -20)    Correct
4: (-20, -1133, 1061)   Correct


In [8]:
getBeaconCount(getInput('example.txt'))

79

In [9]:
getBeaconCount(getInput('input.txt'))

332

In [10]:
def getManhattenDistance(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1]) + abs(a[2] - b[2])


def getLargestManhattenDistance(input):
    points = list(map(lambda x: x[0], calculateRelativePositions(input).values()))
    largest = 0
    for i in range(len(points)):
        for j in range(len(points)):
            current = getManhattenDistance(points[i], points[j])
            if current > largest:
                largest = current
    return largest

In [11]:
getLargestManhattenDistance(getInput('example.txt'))

3621

In [12]:
getLargestManhattenDistance(getInput('input.txt'))

8507